In [1]:
import os

In [2]:
%pwd

'e:\\Data_Science_Project\\TalentGuard\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data_Science_Project\\TalentGuard'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from TalentGuard.constants import *
from TalentGuard.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from TalentGuard import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [20]:
import os
from TalentGuard import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from TalentGuard.config.configuration import DataTransformationConfig
from sklearn.preprocessing import LabelEncoder      



class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def encode_data(self,data):
        """Encode the categorical columns in the DataFrame."""

        label_encoder = LabelEncoder()
        data['salary'] = label_encoder.fit_transform(data['salary'])
        data['Department'] = label_encoder.fit_transform(data['Department'])

        # Save the encoded DataFrame as a .pkl file in the artifacts folder
        encoded_data_path = os.path.join(self.config.root_dir, 'encoded_data.pkl')
        data.to_pickle(encoded_data_path)

        logger.info(f"Encoded data saved to {encoded_data_path}")
        return data

    def train_test_spliting(self):
        # Load the data
        data = pd.read_csv(self.config.data_path)

        # Encode the data
        data = self.encode_data(data)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        # Save the train and test sets as CSV files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print(f"Train shape: {train.shape}")
        print(f"Test shape: {test.shape}")

In [21]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-01-10 23:46:28,090: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-10 23:46:28,095: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-10 23:46:28,101: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-10 23:46:28,105: INFO: common: created directory at: artifacts]
[2025-01-10 23:46:28,107: INFO: common: created directory at: artifacts/data_transformation]
[2025-01-10 23:46:28,160: INFO: 2486909310: Encoded data saved to artifacts/data_transformation\encoded_data.pkl]
[2025-01-10 23:46:28,310: INFO: 2486909310: Split data into training and test sets]
[2025-01-10 23:46:28,314: INFO: 2486909310: Train shape: (11249, 10)]
[2025-01-10 23:46:28,315: INFO: 2486909310: Test shape: (3750, 10)]
Train shape: (11249, 10)
Test shape: (3750, 10)
